# Text Preprocessing

In [1]:
import os

### === Helpers === ###
def perror(msg):
    print("error: " + msg)

def touch(path):
    with open(path, 'a') as f:
        os.utime(path, None) # set access and modified times
        f.close()

def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        return path

    print("path already exists")
    return

## Transcript

In [2]:
# TODO DYNAMIC PATH HANDLE
INPUT_FILE = "test1.mp4"

INPUT_DIR = './input_data'
OUTPUT_DIR = './cache/' # target output for preoprocessing is cache

root, extention  = os.path.splitext(INPUT_FILE)
OUTPUT_FILE = OUTPUT_DIR + root + ".json"

mkdir(OUTPUT_DIR)

path already exists


In [3]:
import whisper_timestamped as whisper
import json

def get_transcript(input_file, output_file):
    audio = whisper.load_audio(input_file)
    model = whisper.load_model("base")

    result = whisper.transcribe(model, audio, language="en")

    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(result, file, indent=2, ensure_ascii=False)

    return result

# Generate transcript if it does not exist in cache
if not os.path.isfile(OUTPUT_FILE):
    try:
        transcript = get_transcript(INPUT_FILE, OUTPUT_FILE)
    except:
        perror("unable to generate transcript")
else: 
    try:
        with open(OUTPUT_FILE, 'r') as f:
            transcript = f.read()   
            transcript = json.loads(transcript)
    except:
        perror("unable to load data from cache")

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



#### Dataframe

In [4]:
import pandas as pd
df = pd.DataFrame(transcript)

def get_text(row):
    return row['segments']['text']

def get_length(row):
    return len(row['segments']['text'])

def get_start(row):
    return row['segments']['start']

def get_end(row):
    return row['segments']['end']

def get_duration(row):
    return round(abs(row['segments']['end'] - row['segments']['start']), 2)

df['text'] = df.apply(get_text,axis=1)
df['text_len'] = df.apply(get_length,axis=1)
df['start'] = df.apply(get_start,axis=1)
df['end'] = df.apply(get_end,axis=1)
df['duration'] = df.apply(get_duration,axis=1)
df

,text,segments,language,text_len,start,end,duration
0,"Well, that's a super powerful idea of generat...","{'id': 0, 'seek': 0, 'start': 3.06, 'end': 11....",en,96,3.06,11.52,8.46
1,for you.,"{'id': 1, 'seek': 0, 'start': 12.26, 'end': 12...",en,9,12.26,12.86,0.60
2,Yeah.,"{'id': 2, 'seek': 0, 'start': 13.78, 'end': 13...",en,6,13.78,13.92,0.14
3,That works to find the best customer for your...,"{'id': 3, 'seek': 0, 'start': 13.92, 'end': 17...",en,53,13.92,17.70,3.78
4,"I mean, to me, advertisement went done well.","{'id': 4, 'seek': 0, 'start': 18.16, 'end': 20...",en,45,18.16,20.64,2.48
...,...,...,...,...,...,...,...
228,then we forget how to do all the stuff that t...,"{'id': 228, 'seek': 85400, 'start': 854.0, 'en...",en,54,854.00,855.94,1.94
229,It's a weird trade off.,"{'id': 229, 'seek': 85400, 'start': 858.16, 'e...",en,24,858.16,859.46,1.30
230,Yeah.,"{'id': 230, 'seek': 85400, 'start': 859.54, 'e...",en,6,859.54,859.76,0.22
231,I agree.,"{'id': 231, 'seek': 85400, 'start': 860.24, 'e...",en,9,860.24,860.70,0.46


## Sentiment

Model: [SamLowe/roberta-base-go_emotions](https://huggingface.co/SamLowe/roberta-base-go_emotions) 

[ONNX Variant](https://huggingface.co/SamLowe/roberta-base-go_emotions-onnx)

Model trained from [roberta-base](https://huggingface.co/roberta-base) on the [go_emotions](https://huggingface.co/datasets/go_emotions) dataset for multi-label classification.


In [5]:
from transformers import pipeline

sentiment_pipe = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

#### Add sentiment analysis to dataframe

In [6]:
def get_sentiment(row):
    return sentiment_pipe(row['text'])[0] # produces a list of dicts for each label

df['sentiment'] = df.apply(get_sentiment, axis=1)

def unpack_sentiment(row):
    sentiment = row['sentiment']
    if sentiment:
        for label_dict in sentiment:
            label = label_dict['label'] + "_sentiment"
            score = label_dict['score']
            row[label] = score
    return row

df = df.apply(unpack_sentiment, axis=1)

def get_positive_sentiment(row):
    res = 0
    res += row['admiration_sentiment']
    res += row['amusement_sentiment']
    res += row['approval_sentiment']
    res += row['caring_sentiment']
    res += row['curiosity_sentiment']
    res += row['desire_sentiment']
    res += row['gratitude_sentiment']
    res += row['joy_sentiment']
    res += row['love_sentiment']
    res += row['optimism_sentiment']
    res += row['pride_sentiment']
    res += row['relief_sentiment']
    res += row['surprise_sentiment']

    return res

def get_negative_sentiment(row):
    res = 0
    res += row['anger_sentiment']
    res += row['annoyance_sentiment']
    res += row['confusion_sentiment']
    res += row['disappointment_sentiment']
    res += row['disapproval_sentiment']
    res += row['disgust_sentiment']
    res += row['embarrassment_sentiment']
    res += row['fear_sentiment']
    res += row['grief_sentiment']
    res += row['nervousness_sentiment']
    res += row['realization_sentiment']
    res += row['remorse_sentiment']
    res += row['sadness_sentiment']
    return res
    
df['positive_sentiment'] = df.apply(get_positive_sentiment, axis=1)
df['negative_sentiment'] = df.apply(get_negative_sentiment, axis=1)

df

,admiration_sentiment,amusement_sentiment,anger_sentiment,annoyance_sentiment,approval_sentiment,caring_sentiment,confusion_sentiment,curiosity_sentiment,desire_sentiment,disappointment_sentiment,...,remorse_sentiment,sadness_sentiment,segments,sentiment,start,surprise_sentiment,text,text_len,positive_sentiment,negative_sentiment
0,0.780682,0.000802,0.000771,0.004517,0.233346,0.001486,0.001338,0.001328,0.001057,0.002207,...,0.000177,0.000493,"{'id': 0, 'seek': 0, 'start': 3.06, 'end': 11....","[{'label': 'admiration', 'score': 0.7806823253...",3.06,0.001670,"Well, that's a super powerful idea of generat...",96,1.050184,0.028494
1,0.001967,0.001697,0.005355,0.012306,0.009886,0.002240,0.002182,0.001814,0.001208,0.002766,...,0.000372,0.002032,"{'id': 1, 'seek': 0, 'start': 12.26, 'end': 12...","[{'label': 'neutral', 'score': 0.9666472077369...",12.26,0.000661,for you.,9,0.025377,0.037557
2,0.003506,0.001027,0.001341,0.005109,0.049398,0.000697,0.002068,0.001384,0.000554,0.001917,...,0.000174,0.000935,"{'id': 2, 'seek': 0, 'start': 13.78, 'end': 13...","[{'label': 'neutral', 'score': 0.9317912459373...",13.78,0.000457,Yeah.,6,0.061778,0.024858
3,0.025332,0.000476,0.000733,0.005156,0.277547,0.017148,0.000957,0.000751,0.003888,0.002212,...,0.000415,0.000736,"{'id': 3, 'seek': 0, 'start': 13.92, 'end': 17...","[{'label': 'neutral', 'score': 0.7166603803634...",13.92,0.000318,That works to find the best customer for your...,53,0.364628,0.035014
4,0.860632,0.000521,0.000901,0.004275,0.153800,0.001880,0.001868,0.001467,0.000969,0.003754,...,0.000293,0.000943,"{'id': 4, 'seek': 0, 'start': 18.16, 'end': 20...","[{'label': 'admiration', 'score': 0.8606323599...",18.16,0.002015,"I mean, to me, advertisement went done well.",45,1.058684,0.032852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.001203,0.001948,0.002421,0.038472,0.011743,0.000528,0.009411,0.000929,0.000846,0.022312,...,0.000678,0.003055,"{'id': 228, 'seek': 85400, 'start': 854.0, 'en...","[{'label': 'neutral', 'score': 0.8300427794456...",854.00,0.002425,then we forget how to do all the stuff that t...,54,0.026062,0.203118
229,0.001949,0.011324,0.022539,0.384109,0.008620,0.000981,0.022400,0.019358,0.002586,0.055946,...,0.001202,0.006487,"{'id': 229, 'seek': 85400, 'start': 858.16, 'e...","[{'label': 'disgust', 'score': 0.4792686104774...",858.16,0.016334,It's a weird trade off.,24,0.067731,1.218652
230,0.003506,0.001027,0.001341,0.005109,0.049398,0.000697,0.002068,0.001384,0.000554,0.001917,...,0.000174,0.000935,"{'id': 230, 'seek': 85400, 'start': 859.54, 'e...","[{'label': 'neutral', 'score': 0.9317912459373...",859.54,0.000457,Yeah.,6,0.061778,0.024858
231,0.014017,0.004272,0.002294,0.011280,0.922422,0.006516,0.006085,0.006843,0.003410,0.002789,...,0.001138,0.002229,"{'id': 231, 'seek': 85400, 'start': 860.24, 'e...","[{'label': 'approval', 'score': 0.922422111034...",860.24,0.001436,I agree.,9,1.002634,0.061051


## Entity Recognition

In [7]:
import spacy

# Run this:
# $ python3 -m spacy download en
#nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_lg")

Labels

In [8]:
# https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf
labels_spacy = """
PERSON                        People, including fictional
NORP                          Nationalities or religious or political groups
FACILITY                      Buildings, airports, highways, bridges, etc.
ORGANIZATION                  Companies, agencies, institutions, etc.
GPE                           Countries, cities, states
LOCATION                      Non-GPE locations, mountain ranges, bodies of water
PRODUCT                       Vehicles, weapons, foods, etc. (Not services)
EVENT                         Named hurricanes, battles, wars, sports events, etc.
WORK OF ART                   Titles of books, songs, etc.
LAW                           Named documents made into laws
LANGUAGE                      Any named language
DATE                          Absolute or relative dates or periods
TIME                          Times smaller than a day
PERCENT                       Percentage (including “%”)
MONEY                         Monetary values, including unit
QUANTITY                      Measurements, as of weight or distance
ORDINAL                       “first”, “second”
CARDINAL                      Numerals that do not fall under another type
"""

label_lookup_table = {
    "PERSON": "People, including fictional",
    "NORP": "Nationalities or religious or political groups",
    "FACILITY": "Buildings, airports, highways, bridges, etc.",
    "ORGANIZATION": "Companies, agencies, institutions, etc.",
    "GPE": "Countries, cities, states",
    "LOCATION": "Non-GPE locations, mountain ranges, bodies of water",
    "PRODUCT": "Vehicles, weapons, foods, etc. (Not services)",
    "EVENT": "Named hurricanes, battles, wars, sports events, etc.",
    "WORK OF ART": "Titles of books, songs, etc.",
    "LAW": "Named documents made into laws",
    "LANGUAGE": "Any named language",
    "DATE": "Absolute or relative dates or periods",
    "TIME": "Times smaller than a day",
    "PERCENT": "Percentage (including “%”)",
    "MONEY": "Monetary values, including unit",
    "QUANTITY": "Measurements, as of weight or distance",
    "ORDINAL": "“first”, “second”",
    "CARDINAL": "Numerals that do not fall under another type"
}

#### Add entities to dataframe

In [9]:
# TODO design decision: handling empty columns, keep or toss

def get_entity_values(row):
    doc = nlp(row['text'])
    # Extract entity details and return as a list of tuples
    return [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]

df['entities'] = df.apply(get_entity_values, axis=1)

df

,admiration_sentiment,amusement_sentiment,anger_sentiment,annoyance_sentiment,approval_sentiment,caring_sentiment,confusion_sentiment,curiosity_sentiment,desire_sentiment,disappointment_sentiment,...,sadness_sentiment,segments,sentiment,start,surprise_sentiment,text,text_len,positive_sentiment,negative_sentiment,entities
0,0.780682,0.000802,0.000771,0.004517,0.233346,0.001486,0.001338,0.001328,0.001057,0.002207,...,0.000493,"{'id': 0, 'seek': 0, 'start': 3.06, 'end': 11....","[{'label': 'admiration', 'score': 0.7806823253...",3.06,0.001670,"Well, that's a super powerful idea of generat...",96,1.050184,0.028494,[]
1,0.001967,0.001697,0.005355,0.012306,0.009886,0.002240,0.002182,0.001814,0.001208,0.002766,...,0.002032,"{'id': 1, 'seek': 0, 'start': 12.26, 'end': 12...","[{'label': 'neutral', 'score': 0.9666472077369...",12.26,0.000661,for you.,9,0.025377,0.037557,[]
2,0.003506,0.001027,0.001341,0.005109,0.049398,0.000697,0.002068,0.001384,0.000554,0.001917,...,0.000935,"{'id': 2, 'seek': 0, 'start': 13.78, 'end': 13...","[{'label': 'neutral', 'score': 0.9317912459373...",13.78,0.000457,Yeah.,6,0.061778,0.024858,[]
3,0.025332,0.000476,0.000733,0.005156,0.277547,0.017148,0.000957,0.000751,0.003888,0.002212,...,0.000736,"{'id': 3, 'seek': 0, 'start': 13.92, 'end': 17...","[{'label': 'neutral', 'score': 0.7166603803634...",13.92,0.000318,That works to find the best customer for your...,53,0.364628,0.035014,[]
4,0.860632,0.000521,0.000901,0.004275,0.153800,0.001880,0.001868,0.001467,0.000969,0.003754,...,0.000943,"{'id': 4, 'seek': 0, 'start': 18.16, 'end': 20...","[{'label': 'admiration', 'score': 0.8606323599...",18.16,0.002015,"I mean, to me, advertisement went done well.",45,1.058684,0.032852,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.001203,0.001948,0.002421,0.038472,0.011743,0.000528,0.009411,0.000929,0.000846,0.022312,...,0.003055,"{'id': 228, 'seek': 85400, 'start': 854.0, 'en...","[{'label': 'neutral', 'score': 0.8300427794456...",854.00,0.002425,then we forget how to do all the stuff that t...,54,0.026062,0.203118,[]
229,0.001949,0.011324,0.022539,0.384109,0.008620,0.000981,0.022400,0.019358,0.002586,0.055946,...,0.006487,"{'id': 229, 'seek': 85400, 'start': 858.16, 'e...","[{'label': 'disgust', 'score': 0.4792686104774...",858.16,0.016334,It's a weird trade off.,24,0.067731,1.218652,[]
230,0.003506,0.001027,0.001341,0.005109,0.049398,0.000697,0.002068,0.001384,0.000554,0.001917,...,0.000935,"{'id': 230, 'seek': 85400, 'start': 859.54, 'e...","[{'label': 'neutral', 'score': 0.9317912459373...",859.54,0.000457,Yeah.,6,0.061778,0.024858,[]
231,0.014017,0.004272,0.002294,0.011280,0.922422,0.006516,0.006085,0.006843,0.003410,0.002789,...,0.002229,"{'id': 231, 'seek': 85400, 'start': 860.24, 'e...","[{'label': 'approval', 'score': 0.922422111034...",860.24,0.001436,I agree.,9,1.002634,0.061051,[]


#### Final Cleanup

In [10]:
# Drop language and segments
df = df.drop(columns='language')
df = df.drop(columns='segments')
df = df.drop(columns='sentiment')

df

,admiration_sentiment,amusement_sentiment,anger_sentiment,annoyance_sentiment,approval_sentiment,caring_sentiment,confusion_sentiment,curiosity_sentiment,desire_sentiment,disappointment_sentiment,...,relief_sentiment,remorse_sentiment,sadness_sentiment,start,surprise_sentiment,text,text_len,positive_sentiment,negative_sentiment,entities
0,0.780682,0.000802,0.000771,0.004517,0.233346,0.001486,0.001338,0.001328,0.001057,0.002207,...,0.000875,0.000177,0.000493,3.06,0.001670,"Well, that's a super powerful idea of generat...",96,1.050184,0.028494,[]
1,0.001967,0.001697,0.005355,0.012306,0.009886,0.002240,0.002182,0.001814,0.001208,0.002766,...,0.000259,0.000372,0.002032,12.26,0.000661,for you.,9,0.025377,0.037557,[]
2,0.003506,0.001027,0.001341,0.005109,0.049398,0.000697,0.002068,0.001384,0.000554,0.001917,...,0.000251,0.000174,0.000935,13.78,0.000457,Yeah.,6,0.061778,0.024858,[]
3,0.025332,0.000476,0.000733,0.005156,0.277547,0.017148,0.000957,0.000751,0.003888,0.002212,...,0.001510,0.000415,0.000736,13.92,0.000318,That works to find the best customer for your...,53,0.364628,0.035014,[]
4,0.860632,0.000521,0.000901,0.004275,0.153800,0.001880,0.001868,0.001467,0.000969,0.003754,...,0.000992,0.000293,0.000943,18.16,0.002015,"I mean, to me, advertisement went done well.",45,1.058684,0.032852,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.001203,0.001948,0.002421,0.038472,0.011743,0.000528,0.009411,0.000929,0.000846,0.022312,...,0.000876,0.000678,0.003055,854.00,0.002425,then we forget how to do all the stuff that t...,54,0.026062,0.203118,[]
229,0.001949,0.011324,0.022539,0.384109,0.008620,0.000981,0.022400,0.019358,0.002586,0.055946,...,0.000993,0.001202,0.006487,858.16,0.016334,It's a weird trade off.,24,0.067731,1.218652,[]
230,0.003506,0.001027,0.001341,0.005109,0.049398,0.000697,0.002068,0.001384,0.000554,0.001917,...,0.000251,0.000174,0.000935,859.54,0.000457,Yeah.,6,0.061778,0.024858,[]
231,0.014017,0.004272,0.002294,0.011280,0.922422,0.006516,0.006085,0.006843,0.003410,0.002789,...,0.004722,0.001138,0.002229,860.24,0.001436,I agree.,9,1.002634,0.061051,[]


### Export

In [11]:
df.to_csv(OUTPUT_DIR + 'out_text_preprocessing.csv', index=False) 

note: summary might be something that is irrelevent until clipped at the end of the pipeline?